In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.simplefilter("ignore")

## Загружаем данные

In [ ]:
rides_info = pd.read_csv("../data/raw/rides_info.csv")
cars = pd.read_csv("../data/raw/car_train.csv")
driver_info = pd.read_csv("../data/raw/driver_info.csv")




## Ознакомимся с данными

In [ ]:
rides_info.head(5)

In [ ]:
rides_info.info()

Наблюдение
- всего в датафрейме 739500 строк
- в столбцах speed_max и user_ride_quality есть пропуски (nan)
- объем занимаемый датафреймом 79.0+ MB

In [ ]:
rides_info.describe().round(2)

Наблюдение
- средняя скорость движения по маршруту по всем поездкам около 47, а максимальная средняя скорость 100. Вероятно, этот столбец имеет размерность км/час
- как минимум в половине случаев во время поездки была одна остановка
- максимальный рейтинг  10, минимум 0. Более половины всех рейтингов с оценками от 3 до 6

In [ ]:
rides_info.describe(include='object')

In [ ]:
rides_info.ride_id.value_counts()[:5]

Наблюдение
- самый часто встречающийся код маршрута H1v и E1y, имеют по 34 поездки

In [ ]:
rides_info.rating.value_counts(bins=4)

Наблюдение
- 50%  поездок с рейтингом ниже 2.5 и выше 5

In [ ]:
rides_info.hist(figsize=(20, 15), layout=(-1, 5));

Делаем выводы о распределении фичей и понимает как именно будем скейлить данные в препроцессинге

Присоединим другие таблицы, чтобы поисследовать относительно таргета

In [ ]:
rides_info = rides_info.merge(cars, on="car_id", how="right")
rides_info = rides_info.merge(driver_info, on="user_id", how="left")

In [ ]:
rides_info.boxplot(
    column=["user_rating"], by="target_class", fontsize=8, figsize=(20, 5)
);

In [ ]:
fig = px.violin(
    rides_info,
    x="target_class",      # категории (тип поломки)
    y="user_rating",       # числовая переменная
    box=True,              # добавить boxplot внутрь скрипки       
    color="target_class",  # окрасить по классам
    hover_data=rides_info.columns  # показывать доп. инфо при наведении
)

fig.update_layout(
    title="Распределение рейтинга водителей по типу поломки",
    xaxis_title="Тип поломки (target_class)",
    yaxis_title="Рейтинг водителя (user_rating)",
    showlegend=False,
    width=1200,
    height=500
)

fig.show()

- Медианы близки (≈7.8–8.2 для всех классов).
- Рейтинг водителя слабый предиктор типа поломки сам по себе.
- engine_ignition / engine_overheat — более широкие “скрипки” и длинные нижние хвосты (больше наблюдений с рейтингом <7).Для этих поломок чаще встречаются водители с низкими оценками → возможная связь с манерой вождения.

In [ ]:
# Корреляционная матрица
corr_matrix = rides_info.select_dtypes(include='number').corr()

# Верхний треугольник (чтобы не дублировалось)
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Размер и стиль
plt.figure(figsize=(14, 10))
sns.set_theme(style="white")

# Рисуем heatmap
sns.heatmap(
    corr_matrix,
    mask=mask,                  # скрыть верхний треугольник
    cmap="coolwarm",            # цветовая палитра
    annot=True,                 # показать значения
    fmt=".2f",                  # округление
    linewidths=0.5,             # разделительные линии
    cbar_kws={"shrink": .8},    # уменьшить colorbar
    square=True,                # клетки квадратные
    annot_kws={"size": 8}       # размер текста
)

# Настройки подписей
plt.title("Корреляционная матрица числовых признаков", fontsize=16, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)

plt.tight_layout()
plt.show()

Наблюдение:
- Сильная положительная корреляция между ride_cost, ride_duration и distance (до 0.9–0.96)  чем длиннее и дольше поездка, тем выше её стоимость.
- Почти полная корреляция между riders и year_to_start (0.99) показывает, что старшие по возрасту автомобили накопили больше поездок 
 эти признаки частично дублируют друг друга.
- Остальные признаки имеют слабую взаимосвязь , что говорит о низкой мультиколлинеарности и потенциальной ценности для модели.
- Заметна умеренная отрицательная корреляция между user_rating и sex (−0.76), что может указывать на различия в оценках водителей по полу  требует проверки.

In [ ]:
g = sns.relplot(
    data=rides_info,
    x="ride_date",
    y="deviation_normal",
    hue="target_class",
    kind="line",  # или scatter
    aspect=4,
)

g.set_xticklabels(rotation=45, horizontalalignment="right", step=2);

Признак deviation_normal потенциально информативен — его динамика отличается в зависимости от типа поломки (target_class).
Следовательно, его изменение или рост может быть предиктором (feature) в модели прогнозирования типа поломки.

#TODO Возможно стоит сделать фичу типа изменение во времени этой фичи - типа производная 

In [ ]:
g = sns.relplot(
    data=rides_info,
    x="ride_date",
    y="user_ride_quality",
    kind="line",
    hue="target_class",
    aspect=4,
)

g.set_xticklabels(rotation=45, horizontalalignment="right", step=2);

- Тренд по типам поломок различается.
Например, зелёная линия (gear_stick) и синяя (another_bug) лежат выше других → это значит, что водители этих машин ездили спокойнее (высокое качество вождения).
- Некоторые типы поломок (“engine_fuel”, “engine_ignition”, “engine_overheat”) находятся ниже,
что говорит о более агрессивной манере езды или нестабильных скоринговых показателях у водителей таких машин.

Тип манеры вождения (user_ride_quality) статистически различается между группами поломок.
Более агрессивная езда (низкий user_ride_quality) чаще наблюдается у машин с поломками двигателя и перегревом.
Это подтверждает гипотезу, что стиль вождения может служить ранним предиктором технических неисправностей.

In [ ]:
g = sns.relplot(
    data=rides_info,
    x="ride_duration",
    y="ride_cost",
    hue="target_class",
    
    kind="scatter",
    aspect=4,
    alpha=0.5,
);

Возможно feature = ride_cost/ride_duration - это отличный доп.признак

Посмотрим на распределение таргета

In [ ]:
ax = sns.countplot(data=rides_info, y='target_class', order=rides_info['target_class'].value_counts().index)
ax.set_title('Class balance'); plt.show()




Видим, что есть дисбаланс и надо это учитывать

Посмторим также реграссионный таргет и попробуем его выпрямить логарифмом

In [ ]:
fig, axes = plt.subplots(1,2, figsize=(12,4))
sns.histplot(rides_info['target_reg'], ax=axes[0], bins=50)
sns.histplot(np.log1p(rides_info['target_reg']), ax=axes[1], bins=50)
axes[0].set_title('target_reg'); axes[1].set_title('log1p(target_reg)');

In [ ]:
rides_info["ride_date"] = pd.to_datetime(rides_info["ride_date"], errors="coerce")
rides_info["tariff"] = rides_info["ride_cost"] / (rides_info["ride_duration"] + 1e-9)  # простая фича: цена за единицу времени

In [ ]:
plt.figure(figsize=(12,4))
sns.boxplot(data=rides_info, x="target_class", y="deviation_normal")
plt.xticks(rotation=30, ha="right")
plt.title("deviation_normal по target_class")
plt.show()

Видим что в среднем при всех поломках отклонение от нормы одинаковое и много выбросов

In [ ]:
plt.figure(figsize=(6,5))
sns.scatterplot(data=rides_info.sample(5000, random_state=42), x="speed_avg", y="speed_max", alpha=0.4)
plt.title("speed_max vs speed_avg (сэмпл 5k)")
plt.show()

In [ ]:
ct = pd.crosstab(rides_info["car_type"], rides_info["target_class"])
plt.figure(figsize=(10,6))
sns.heatmap(ct, annot=False, cmap="Blues")
plt.title("car_type × target_class (частоты)")
plt.show()

Большинство поломок приходится на автомобили класса economy — вероятно, из-за высокой нагрузки и интенсивного использования.
Машины классов business и premium ломаются заметно реже

In [ ]:
daily = rides_info.groupby("ride_date", as_index=False)["deviation_normal"].mean()
plt.figure(figsize=(12,4))
sns.lineplot(data=daily, x="ride_date", y="deviation_normal")
plt.title("Средний deviation_normal по дням")
plt.show()

Показатель deviation_normal хорошо отражает общий технический тренд автопарка.
Видно циклическое ухудшение и восстановление состояния машин, что может соответствовать периодическим ремонтам.
Этот признак потенциально полезен для предсказания момента поломки (временные лаги или производные по времени).

In [ ]:
rides_info = pd.read_csv("../data/raw/rides_info.csv")
print(rides_info.shape)
rides_info.hist(figsize=(25, 4), layout=(2, 5), bins=40)
rides_info.sample(3)

In [ ]:
fix_info = pd.read_csv("../data/raw/fix_info.csv").sort_values('worker_id')
print(fix_info.shape)
fix_info.hist(figsize=(12, 2))
fix_info.sample(3)

In [ ]:
train = pd.read_csv("../data/raw/car_train.csv")
print(train.shape)
train.hist(figsize=(25, 4), layout=(2, 5), bins=30)
train.sample(3)

Поисследуем признаки

In [ ]:

fix_info_gr = fix_info.groupby("car_id", as_index=False).agg(
    
    # Все встроенные статистики
    worker_count=("worker_id", "count"),
    work_duration_mean=("work_duration", "mean"),
    work_duration_max=("work_duration", "max"),
    destroy_degree_std=("destroy_degree", "std"),
    destroy_degree_sum=("destroy_degree", "sum"))

In [ ]:
tmp = train.merge(fix_info_gr, on="car_id", how="left")
tmp.head(3)

In [ ]:
import seaborn as sns

g = sns.displot(
    data=tmp,
    x="destroy_degree_sum",
    y="riders",
    aspect=2,
    kind="hist",
    alpha=0.8,
    hue="target_class"
).set_xticklabels(rotation=45, horizontalalignment="right")

In [ ]:
g = sns.displot(
    data=tmp,
    x="destroy_degree_std",
    y="riders",
    aspect=2,
    kind="hist",
    alpha=0.8,
    hue="target_class"
).set_xticklabels(rotation=45, horizontalalignment="right");

•	Для части машин наблюдается низкое отклонение (около 2) → поломки однотипные (часто мелкие, повторяющиеся). Вохможно это хорошая фича то есть эта фича хорошо разделяет engine_check


In [ ]:
rides_info = pd.read_csv("../data/raw/rides_info.csv")
cars = pd.read_csv("../data/raw/car_train.csv")
driver_info = pd.read_csv("../data/raw/driver_info.csv")
rides_info = rides_info.merge(cars, on="car_id", how="right")
rides_info = rides_info.merge(driver_info, on="user_id", how="left")

rides_info["ride_date"] = pd.to_datetime(rides_info["ride_date"], errors="coerce")
rides_info["tariff"] = rides_info["ride_cost"] / (rides_info["ride_duration"] + 1e-9)  # простая фича: цена за единицу времени

In [ ]:
cols = ["speed_avg","speed_max","tariff","user_ride_quality","deviation_normal","target_class"]
sns.pairplot(rides_info[cols].dropna().sample(4000, random_state=7), hue="target_class", diag_kind="hist")
plt.show()

В целом не наблюдается мультиколлинеарности

In [ ]:
sns.displot(rides_info, x="target_reg", col="target_class", col_wrap=3, height=2.7, bins=40)
plt.suptitle("Распределение времени до поломки по классам", y=1.02); plt.show()

Для всех типов поломок распределение времени до отказа  имеет ярко выраженный правый хвост: большинство машин ломаются относительно быстро, а случаи с большим временем до поломки встречаются редко. Формы распределений по классам схожи, что говорит о слабой дискриминирующей способности этого признака. Для моделей регрессии целесообразно применить логарифмирование (target_reg) для стабилизации дисперсии и уменьшения влияния выбросов.

In [ ]:
top_models = (train["model"].value_counts().head(10).index)
tmp = train[train["model"].isin(top_models)]

fig = px.bar(tmp.groupby(["model","target_class"]).size().reset_index(name="cnt"),
             x="model", y="cnt", color="target_class", barmode="stack",
             title="Топ-10 моделей × распределение поломок")
fig.show()

Видно, что доли классов engine_check и engine_ignition стабильно высоки для всех моделей, однако у отдельных (например, VW Polo VI) выше доля engine_overheat. Это может указывать на специфику конструкции или условий эксплуатации.

In [ ]:
plt.figure(figsize=(6,5))
sns.scatterplot(data=rides_info.sample(8000, random_state=1), x="user_time_accident", y="user_ride_quality", alpha=0.4)
plt.title("Аварийность vs качество езды"); plt.show()

 видна слабая отрицательная связь между числом инцидентов и качеством езды: водители с низкими скоринговыми оценками  чаще имеют больше зафиксированных аварий. Основное облако точек сосредоточено в зоне 0–15 инцидентов. Таким образом, показатель user_ride_quality можно рассматривать как ранний индикатор рискованного стиля вождения, потенциально влияющего на вероятность поломок автомобиля.

In [ ]:
q = pd.qcut(rides_info["deviation_normal"], q=10)
tab = pd.crosstab(q, rides_info["target_class"], normalize="index")
plt.figure(figsize=(10,4)); sns.heatmap(tab, cmap="mako"); plt.title("Доли классов по бинам deviation_normal"); plt.show()

Вблизи нулевых значений  выделяется electro_bug — электрические неисправности возникают при стабильных механических характеристиках.
При сильных отрицательных отклонениях  чаще встречаются поломки типа break_bug и gear_stick, что указывает на связь этих дефектов с ухудшением общих параметров автомобиля.
При положительных и высоких отклонениях  чаще фиксируются engine_ignition и engine_fuel, что может отражать перегрузку двигателя или экстремальные режимы работы.

Таким образом, признак deviation_normal является крайне информативным

In [ ]:
fix_info = pd.read_csv("../data/raw/fix_info.csv")
agg = fix_info.groupby("work_type")["destroy_degree"].mean().reset_index()
sns.barplot(data=agg, x="work_type", y="destroy_degree")
plt.xticks(rotation=20, ha="right"); plt.title("Средний износ по типам работ"); plt.show()

Наибольшие значения наблюдаются для категории repair, что отражает выполнение ремонта при серьёзных повреждениях.
Остальные типы работ связаны с профилактическими или лёгкими вмешательствами (например, заправка, смена масла) и сопровождаются низким уровнем износа.